<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/NLP/TTS_AudioBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tactron 2

In [1]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS
os.chdir("TensorFlowTTS")
!pip install  .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 4956 (delta 1), reused 1 (delta 0), pack-reused 4945
Receiving objects: 100% (4956/4956), 81.44 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (3153/3153), done.
Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 51kB/s 
     |████████████████████████████████| 1.1MB 52.2MB/s 
     |████████████████████████████████| 184kB 54.9MB/s 
     |████████████████████████████████| 245kB 50.3MB/s 
     |████████████████████████████████| 225kB 52.4MB/s 
     |████████████████████████████████| 788kB 43.8MB/s 
     |████████████████████████████████| 1.7MB 44.5MB/s 
     |████████████████████████████████| 3.1MB 46.6MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 184kB 54.4MB/s 
  Created wheel for TensorFlowTTS: filename=TensorFlowTTS-0.0-cp3

In [2]:
print("Downloading Tacotron2 model...")
!gdown --id {"12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW"} -O tacotron2-120k.h5
!gdown --id {"1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY"} -O tacotron2_config.yml

Downloading...
From: https://drive.google.com/uc?id=12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW
To: /content/tacotron2-120k.h5
128MB [00:01, 81.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY
To: /content/tacotron2_config.yml
100% 1.49k/1.49k [00:00<00:00, 2.76MB/s]


In [3]:
print("Downloading Multi-band MelGAN model...")
!gdown --id {"1kChFaLI7slrTtuk3pvcOiJwJDCygsw9C"} -O mb.melgan-940k.h5
!gdown --id {"1YC_kZpuRZGQ-JfMKj1LC0YRyKXsgLTJL"} -O mb.melgan_config.yml

Downloading...
From: https://drive.google.com/uc?id=1kChFaLI7slrTtuk3pvcOiJwJDCygsw9C
To: /content/mb.melgan-940k.h5
10.2MB [00:00, 38.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YC_kZpuRZGQ-JfMKj1LC0YRyKXsgLTJL
To: /content/mb.melgan_config.yml
100% 2.18k/2.18k [00:00<00:00, 1.92MB/s]


In [4]:
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [5]:
tacotron2_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="tacotron2-120k.h5",
    training=False, 
    name="tacotron2"
)

In [6]:
mb_melgan_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/multiband_melgan/conf/multiband_melgan.v1.yaml')
mb_melgan = TFAutoModel.from_pretrained(
    config=mb_melgan_config,
    pretrained_path="mb.melgan-940k.h5",
    name="mb_melgan"
)

In [7]:
print("Downloading ljspeech_mapper.json ...")
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

Downloading...
From: https://drive.google.com/uc?id=1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e
To: /content/ljspeech_mapper.json
100% 3.57k/3.57k [00:00<00:00, 5.86MB/s]


In [8]:
processor = AutoProcessor.from_pretrained(pretrained_path="./ljspeech_mapper.json")

In [9]:
def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH":
    mel_before, mel_outputs, duration_outputs = text2mel_model.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  elif vocoder_name == "MB-MELGAN":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()


In [10]:
import re
with open('/content/final.txt', 'r') as reader:
    chapter = reader.read()

chapter=re.sub('\n\n', '\n', chapter)
chapter=re.sub('\t', '', chapter)
chapter=re.sub('—', '', chapter)
chapter_list=list(chapter.split("\n")) 
chapter_list=list(filter(None, chapter_list))

In [11]:
len(chapter_list)

4932

In [12]:
import soundfile as sf

In [ ]:
from tqdm import tqdm
final=[]
for i,j in enumerate(tqdm(chapter_list[])):
  #print((i))
  _, _, audios = do_synthesis(j, tacotron2, mb_melgan, "TACOTRON", "MB-MELGAN")
  sf.write("sample%04d.ogg" %i, audios, 22050)
  final.append("file '/content/sample%04d.ogg'" %i)

In [ ]:
#ipd.Audio(final[1], rate=22050)

In [38]:
with open("outfile", "w") as outfile:
    outfile.write("\n".join(final))

In [39]:
!ffmpeg -f concat -safe 0 -i outfile final.ogg

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [40]:
!cp ./final.ogg /content/drive/MyDrive

In [41]:
!rm -rf sample*.ogg

## Flowtron

In [51]:
!gdown --id {"1Cjd6dK_eFz6DE0PKXKgKxrzTUqzzUDW-"} -O flowtron_ljs.pt

Downloading...
From: https://drive.google.com/uc?id=1Cjd6dK_eFz6DE0PKXKgKxrzTUqzzUDW-
To: /content/flowtron_ljs.pt
244MB [00:02, 109MB/s]


In [52]:
!gdown --id {"1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF"} -O waveglow_256channels_universal_v5.pt

Downloading...
From: https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
To: /content/waveglow_256channels_universal_v5.pt
676MB [00:08, 76.7MB/s]


In [56]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 12.8MB/s 


In [71]:
!git clone https://github.com/NVIDIA/flowtron.git

Cloning into 'flowtron'...
remote: Enumerating objects: 135, done.
remote: Total 135 (delta 0), reused 0 (delta 0), pack-reused 135
Receiving objects: 100% (135/135), 2.79 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [72]:
os.chdir("flowtron")
!git clone https://github.com/NVIDIA/tacotron2.git

Cloning into 'tacotron2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 406 (delta 0), reused 0 (delta 0), pack-reused 403
Receiving objects: 100% (406/406), 2.70 MiB | 4.93 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [73]:
!git clone https://github.com/NVIDIA/waveglow.git

Cloning into 'waveglow'...
remote: Enumerating objects: 190, done.
remote: Total 190 (delta 0), reused 0 (delta 0), pack-reused 190
Receiving objects: 100% (190/190), 435.59 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [75]:
!mv /content/flowtron/waveglow/* /content/flowtron/tacotron2/waveglow

In [89]:
!python inference.py -c config.json -f /content/flowtron_ljs.pt -w /content/waveglow_256channels_universal_v5.pt -t "He assumed she was a former French spook, either DGSE or DCRI, although she also could have been from DST, which became DCRI in 2008. Virtually everyone Hazelton met with in the course of his work was a former intelligence officer, so this was no stretch." -i 0

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or